In [ ]:
# train long
from mine import *
from long import *
from helper import *
from utils import *

start = 1550
end = 1750

env = Enviroment(stocks = ['AAPL'],start = start, end = end, lookback = 10, variance = 0.1,cluster = c)

long_agent = DQN_Agent_Adapted(path_to_save,gamma,max_mem_size,C,batch_size,epsilon, eps_min, eps_dec, num_layers,hidden_size,added_size,fc1_dims,fc2_dims,lr,wd,lookback,x2_size,scheduler_max_value,scheduler_initial,mc)
long_agent.load_weights(name = '1500_1700')
long_agent.load_buffer()

reward_running_count_finder = long
long_agent.directory = r"D:\\Project\\src\\Test_Policies\\long\\"
circle = 27

while end <= 2870:
    
    # train kmeans
    c.train(start,end)
    
    for it in range(itterations):
        stock = np.random.choice(stocks)
        env.change_and_reset(stocks = [stock])

        running_count = 0
        list_of_buy_days = []

        treward = 0
        for day in range(env.start,env.end):

            done = day == env.end-1

            x1 = env.feature_matrix(stock)
            x2 = [running_count] + env.GOAT_features(stock)
           
            long_agent.Q_eval.eval()
            with T.no_grad():
                action = long_agent.choose_action(x1,x2)
            long_agent.Q_eval.train()
            
            reward,running_count,list_of_buy_days = reward_running_count_finder(env,action,running_count,list_of_buy_days)
            if done and running_count !=0:
                done_reward,running_count,list_of_buy_days = reward_running_count_finder(env,0,running_count,list_of_buy_days)
                reward = reward + done_reward
                
            treward += reward
            x1_next = env.feature_matrix(stock)
            x2_next = [running_count] + env.GOAT_features(stock)
            
            long_agent.store_transition(x1,x2,action,reward,x1_next,x2_next,done,circle)
            long_agent.learn()
            
        env.reset()
        #print(it)
        
    long_agent.save_weights(name = f'{start}_{end}')
    
    start += 50
    end += 50
    circle += 1

In [45]:
from mine import *

# train short
from short import *
from helper import *

long_agent = DQN_Agent_Adapted(path_to_save,gamma,max_mem_size,C,batch_size,epsilon, eps_min, eps_dec, num_layers,hidden_size,added_size,fc1_dims,fc2_dims,lr,wd,lookback,x2_size,scheduler_max_value,scheduler_initial,mc)
long_agent.load_weights(name = '1500_1700')
long_agent.load_buffer()

reward_running_count_finder = short
long_agent.directory = r"D:\\Project\\src\\Test_Policies\\short\\"
circle = 27

while end <= 2870:
    
    # train kmeans
    c.train(start,end)
    
    for it in range(itterations):
        stock = np.random.choice(stocks)
        env.change_and_reset(stocks = [stock])

        running_count = 0
        list_of_buy_days = []

        treward = 0
        for day in range(env.start,env.end):

            done = day == env.end-1

            x1 = env.feature_matrix(stock)
            x2 = [running_count] + env.GOAT_features(stock)
           
            long_agent.Q_eval.eval()
            with T.no_grad():
                action = long_agent.choose_action(x1,x2)
            long_agent.Q_eval.train()
            
            reward,running_count,list_of_buy_days = reward_running_count_finder(env,action,running_count,list_of_buy_days)
            if done and running_count !=0:
                done_reward,running_count,list_of_buy_days = reward_running_count_finder(env,0,running_count,list_of_buy_days)
                reward = reward + done_reward
                
            treward += reward
            x1_next = env.feature_matrix(stock)
            x2_next = [running_count] + env.GOAT_features(stock)
            
            long_agent.store_transition(x1,x2,action,reward,x1_next,x2_next,done,circle)
            long_agent.learn()
            
        env.reset()
        #print(it)
        
    long_agent.save_weights(name = f'{start}_{end}')
    
    start += 50
    end += 50
    circle += 1

In [1]:
from mine import *
from long import *

# find policy long
print(path_to_save)
path = 'D:\\Project\\src\\Test_Policies\\final_policy\\'

start = 1750
end = 1800

file_start = start -200
file_end = start

agent = DQN_Agent_Adapted(path_to_save,gamma,max_mem_size,C,batch_size,epsilon, eps_min, eps_dec, num_layers,hidden_size,added_size,fc1_dims,fc2_dims,lr,wd,lookback,x2_size,scheduler_max_value,scheduler_initial,mc)
agent.directory = "D:\\Project\\src\\Test_Policies\\long\\"
policy = np.zeros((1100,29))

while end <= 2850:

    c.train(file_start,file_end)

    env = Enviroment(start = start, end = end, variance = 0, lookback = lookback,cluster = c,cash = 1e6,p_finder = shares)
    agent.load_weights(name = f'{file_start}_{file_end}')

    running_counts = [0 for _ in range(len(stocks))]
    for day in range(start,end):
        day_policy = [0 for _ in range(len(stocks))]

        for number,stock in enumerate(stocks):
            running_count = running_counts[number]
            x1 = env.feature_matrix(stock)
            x2 = [running_count] + env.GOAT_features(stock)

            agent.Q_eval.eval()
            with T.no_grad():
                action = agent.choose_action(x1,x2)
            agent.Q_eval.train()

            if action == 2: running_counts[number] = running_counts[number] + 1
            if action == 0: running_counts[number] = 0

            day_policy[number] = action

        policy[day - 1750,:] = day_policy
        env.step(running_counts)

    start += switch
    end += switch
    
np.save(path + f'long.npy',policy)

started_training
D:\Project\src\Weights\SSA\long_final


In [1]:
from mine import *
from short import *

# find policy short

path = 'D:\\Project\\src\\Test_Policies\\final_policy\\'

start = 1750
end = 1800

file_start = start -200
file_end = start

agent = DQN_Agent_Adapted(path_to_save,gamma,max_mem_size,C,batch_size,epsilon, eps_min, eps_dec, num_layers,hidden_size,added_size,fc1_dims,fc2_dims,lr,wd,lookback,x2_size,scheduler_max_value,scheduler_initial,mc)
agent.directory = "D:\\Project\\src\\Test_Policies\\short\\"
policy = np.zeros((1100,29))

while end <= 2850:

    c.train(file_start,file_end)

    env = Enviroment(start = start, end = end, variance = 0, lookback = lookback,cluster = c,cash = 1e6,p_finder = shares)
    agent.load_weights(name = f'{file_start}_{file_end}')

    running_counts = [0 for _ in range(len(stocks))]
    for day in range(start,end):
        day_policy = [0 for _ in range(len(stocks))]

        for number,stock in enumerate(stocks):
            running_count = running_counts[number]
            x1 = env.feature_matrix(stock)
            x2 = [running_count] + env.GOAT_features(stock)

            agent.Q_eval.eval()
            with T.no_grad():
                action = agent.choose_action(x1,x2)
            agent.Q_eval.train()

            if action == 2: running_counts[number] = running_counts[number] + 1
            if action == 0: running_counts[number] = 0

            day_policy[number] = action

        policy[day - 1750,:] = day_policy
        env.step([-i for i in running_counts])

    start += switch
    end += switch
    
np.save(path + f'short.npy',policy)

started_training
